# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: distillBERT
* Evaluation approach: evaluate method from HuggingFace Trainer class
* Fine-tuning dataset: glue-mrpc

In [1]:
pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 36.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
from datasets import load_dataset

ds = load_dataset("nyu-mll/glue", "mrpc")
ds

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [14]:
uL = set(ds["train"]["label"])
uL

{0, 1}

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding


tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
md = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2)

#def tokenizing(examples):
    #return tokenizer(examples["sentence1"], examples["sentence2"], truncation = True, padding = 'max_length' , max_length = 128)
def tokenizing(examples):
    # Print examples and tokenized output to debug
    tokenized = tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding='max_length', max_length=128)
    #print(tokenized)  # Print to verify structure
    return tokenized

tokenized_dataset = ds.map(tokenizing, batched= True)
tokenized_dataset =tokenized_dataset.rename_column('label', 'labels')
data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
data_collator


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [6]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {'accuracy': accuracy_score(p.label_ids, preds)}


sTraining_Arguments = TrainingArguments("sTraining")
sTrainer = Trainer(
    model = md, 
    args = sTraining_Arguments,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    compute_metrics = compute_metrics,
    data_collator = data_collator
)

res = sTrainer.evaluate()
res

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6881133317947388,
 'eval_accuracy': 0.5857843137254902,
 'eval_runtime': 2.1878,
 'eval_samples_per_second': 186.492,
 'eval_steps_per_second': 23.311}

In [13]:
md

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
from peft import LoraConfig, get_peft_model

configuration = LoraConfig(
    task_type='SEQ_CLS'
    ,r = 16
    , lora_alpha = 32
    , lora_dropout= 0.1
    ,target_modules = [
        "transformer.layer.*.attention.q_lin"
        ,"transformer.layer.*.attention.k_lin"
        ,"transformer.layer.*.attention.v_lin"]
)
loraMD = get_peft_model(md, configuration)
loraMD.print_trainable_parameters()

trainable params: 442,368 || all params: 67,989,508 || trainable%: 0.650641566637017


In [15]:
from transformers import Trainer, TrainingArguments

trainingArgs = TrainingArguments(
    output_dir = "./savingTrainingArgsSUBMISSION" #where should i keep this?
    ,evaluation_strategy = "epoch"
    ,learning_rate = 5e-5
    ,per_device_train_batch_size=16
    ,per_device_eval_batch_size=16
    ,num_train_epochs=3 
    ,weight_decay=0.01
)

#def debug_collator3(features):
    #print("Features received by collator:")
    #for feature in features:
        #print(feature)
    #return data_collator(features)



trn = Trainer(
    model = loraMD
    ,args = trainingArgs
    ,train_dataset = tokenized_dataset["train"]
    ,eval_dataset = tokenized_dataset["validation"]
    ,compute_metrics = compute_metrics
    ,data_collator = data_collator
)

#print(tokenized_dataset['train'][0]['input_ids'])  # Check input_ids
es = trn.train()
es

loraMD.save_pretrained("./savingloraMDafterTrainingSUBMISSION")
tokenizer.save_pretrained("./savingloraMDafterTrainingSUBMISSION")


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.450080,0.799020
2,No log,0.420984,0.796569
3,0.435100,0.426960,0.801471


Checkpoint destination directory ./savingTrainingArgsSUBMISSION/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


('./savingloraMDafterTrainingSUBMISSION/tokenizer_config.json',
 './savingloraMDafterTrainingSUBMISSION/special_tokens_map.json',
 './savingloraMDafterTrainingSUBMISSION/vocab.txt',
 './savingloraMDafterTrainingSUBMISSION/added_tokens.json',
 './savingloraMDafterTrainingSUBMISSION/tokenizer.json')

In [18]:
print(tokenized_dataset.column_names)


{'train': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'], 'validation': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask'], 'test': ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'attention_mask']}


In [31]:
print(tokenized_dataset["train"].features)
print(tokenized_dataset["train"][0])  

{'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(names=['not_equivalent', 'equivalent'], id=None), 'idx': Value(dtype='int32', id=None), 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}
{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0, 'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
from datasets import load_dataset

peftDS = load_dataset("nyu-mll/glue", "mrpc")


In [19]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer

InferenceTokenizer = AutoTokenizer.from_pretrained("./savingloraMDafterTrainingSUBMISSION")
InferenceLora_Model = AutoPeftModelForSequenceClassification.from_pretrained("./savingloraMDafterTrainingSUBMISSION")


def tokenizingPEFT(examplesPEFT):
    tokenized = InferenceTokenizer(examplesPEFT["sentence1"], examplesPEFT["sentence2"], truncation=True, padding='max_length', max_length=128)
    #print(tokenized)  
    return tokenized

tokenized_datasetPEFT = peftDS.map(tokenizingPEFT, batched= True)
#
data_collatorPEFT = DataCollatorWithPadding(tokenizer = InferenceTokenizer)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metricsPEFT(p):
    preds = np.argmax(p.predictions, axis=1)
    return {'accuracy': accuracy_score(p.label_ids, preds)}


peftTraining_Arguments = TrainingArguments("peftEvalArgs")
peftTrainer = Trainer(
    model = InferenceLora_Model, 
    args = peftTraining_Arguments,
    train_dataset = tokenized_datasetPEFT["train"],
    eval_dataset = tokenized_datasetPEFT["validation"],
    compute_metrics = compute_metricsPEFT,
    data_collator = data_collatorPEFT
)

peftEVAL = peftTrainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [21]:
print("Original Model Accuracy:", res['eval_accuracy'])
print("PEFT Model Accuracy:", peftEVAL['eval_accuracy'])

Original Model Accuracy: 0.5857843137254902
PEFT Model Accuracy: 0.8014705882352942
